# PyCity Schools Analysis

* As a whole, top 5 performing schools were all charter schools which had comparable reading and math passing percentages, while bottom 5 performing schools were district schools that had significantly lower math passing percentages than reading passing percentages.

* There appears to be no significant difference by grade in average reading and math scores, although all are consistently higher at charter schools.


### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
total_schools = len(school_data["School ID"])
total_students= len(school_data_complete["Student ID"])
total_budget = school_data["budget"].sum()
average_math = round(school_data_complete["math_score"].sum()/total_students * 2 / 2)
average_read = round(school_data_complete["reading_score"].sum()/total_students * 2 / 2)
overall_avg_score = round((school_data_complete["reading_score"].sum()+school_data_complete["math_score"].sum())/total_students/2  * 2 / 2)
math_pass_table = student_data[["Student ID","math_score"]]
math_passing = math_pass_table.loc[math_pass_table["math_score"] > 69, ["Student ID"]]
math_passing_count = math_passing["Student ID"].count()
math_passing_pct = round(math_passing_count/total_students * 100 * 2 / 2)
read_pass_table = student_data[["Student ID","reading_score"]]
read_passing = read_pass_table.loc[read_pass_table["reading_score"] > 69, ["Student ID"]]
read_passing_count = read_passing["Student ID"].count()
read_passing_pct = round(read_passing_count/total_students * 100 * 2 / 2)
comb_passing_pct = round((((math_passing_count + read_passing_count)/(total_students+total_students))) * 100 * 2 / 2)
#Place the data series into a new column inside of the DataFrame
district_summary = pd.DataFrame.from_dict({"total_schools":[total_schools], "total_students":[total_students], "total_budget":[total_budget],
                                          "average math score":[average_math], "average reading score":[average_read], "overall passing rate":[overall_avg_score],
                                          "percent of students passing math":[math_passing_pct], "percent of students passing reading":[read_passing_pct], 
                                          "percent of students overall passing":[comb_passing_pct]})
district_summary["total_budget"] = district_summary["total_budget"].map('${:,}'.format)
district_summary["total_students"] = district_summary["total_students"].map('{:,}'.format)
district_summary

,total_schools,total_students,total_budget,average math score,average reading score,overall passing rate,percent of students passing math,percent of students passing reading,percent of students overall passing
0,15,"39,170","$24,649,428",79.0,82.0,80.0,75.0,86.0,80.0


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:
# School Summmary
# Get school type and budget
school_data_complete.head()

school_table1 = school_data_complete.loc[school_data_complete["budget"] > 0, ["school_name", "type" , "size", "budget"]]
school_table1["size"]=school_table1["size"].astype(int)
grouped_school_table1=school_table1.groupby('school_name').min()

# Average math and reading scores
school_table2 = school_data_complete.loc[school_data_complete["math_score"] > 0, ["school_name", "math_score", "reading_score"]]
grouped_school_table2 = school_table2.groupby(["school_name"]).sum()


# merge school type and budget with math and reading scores
school_merge = grouped_school_table1.merge(grouped_school_table2, on="school_name", how="outer")
school_merge["average_math"] = school_merge["math_score"]/school_merge["size"] 
school_merge["average_read"] = school_merge["reading_score"]/school_merge["size"]
school_merge["budget_per_student"] = round(school_merge["budget"]/school_merge["size"] * 2 / 2)


# students passing math by school
school_table3 = school_data_complete.loc[school_data_complete["math_score"] > 69, ["school_name","grade"]]
grouped_school_table3 = school_table3.groupby(["school_name"]).count()
# of students passing reading per school
school_table4 = school_data_complete.loc[school_data_complete["reading_score"] > 69, ["school_name","grade"]]
grouped_school_table4 = school_table4.groupby(["school_name"]).count()
school_merge_passing = grouped_school_table3.merge(grouped_school_table4, on="school_name", how="outer")


# final merge passing school data with total school data to produce school summary
school_merge2 = school_merge.merge(school_merge_passing, on="school_name", how="outer").rename(columns={"grade_x": "math_passing_students","grade_y": "reading_passing_students"})
school_merge2 = school_merge2.drop(columns=["math_score","reading_score"])
school_merge2["math_passing_pct"] = round(school_merge2["math_passing_students"]/school_merge2["size"] * 100 * 2 / 2)
school_merge2["reading_passing_pct"] = round(school_merge2["reading_passing_students"]/school_merge2["size"] * 100 * 2 / 2)
school_merge2["overall_passing_pct"] = round(school_merge2["reading_passing_students"]/(school_merge2["size"] * 2) * 100 * 2 / 2)
school_merge2 = school_merge2.drop(columns=["math_passing_students","reading_passing_students"])
school_merge2["budget"]=school_merge2["budget"].map('${:,}'.format)
school_merge2

,type,size,budget,average_math,average_read,budget_per_student,math_passing_pct,reading_passing_pct,overall_passing_pct
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928",77.048432,81.033963,628.0,67.0,82.0,41.0
Cabrera High School,Charter,1858,"$1,081,356",83.061895,83.975780,582.0,94.0,97.0,49.0
Figueroa High School,District,2949,"$1,884,411",76.711767,81.158020,639.0,66.0,81.0,40.0
Ford High School,District,2739,"$1,763,916",77.102592,80.746258,644.0,68.0,79.0,40.0
Griffin High School,Charter,1468,"$917,500",83.351499,83.816757,625.0,93.0,97.0,49.0
Hernandez High School,District,4635,"$3,022,020",77.289752,80.934412,652.0,67.0,81.0,40.0
Holden High School,Charter,427,"$248,087",83.803279,83.814988,581.0,93.0,96.0,48.0
Huang High School,District,2917,"$1,910,635",76.629414,81.182722,655.0,66.0,81.0,41.0
Johnson High School,District,4761,"$3,094,650",77.072464,80.966394,650.0,66.0,81.0,41.0


In [4]:
# Make a copy to work with for each bins exercise.  Not the most efficient idea, but it gets the job done.
school_merge2_copy1 = school_merge2.copy()
school_merge2_copy2 = school_merge2.copy()
school_merge2_copy3 = school_merge2.copy()

## Top Performing Schools (By Passing Rate)

In [5]:
# Sort data in ascending order by overall passing rate
school_merge2.sort_values(by=['overall_passing_pct'], inplace=True, ascending=False)
#Pick top 5
school_merge2[0:5]

,type,size,budget,average_math,average_read,budget_per_student,math_passing_pct,reading_passing_pct,overall_passing_pct
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",83.061895,83.975780,582.0,94.0,97.0,49.0
Griffin High School,Charter,1468,"$917,500",83.351499,83.816757,625.0,93.0,97.0,49.0
Thomas High School,Charter,1635,"$1,043,130",83.418349,83.848930,638.0,93.0,97.0,49.0
Holden High School,Charter,427,"$248,087",83.803279,83.814988,581.0,93.0,96.0,48.0
Pena High School,Charter,962,"$585,858",83.839917,84.044699,609.0,95.0,96.0,48.0


## Bottom Performing Schools (By Passing Rate)

In [6]:
# Sort data in descending order by overall passing rate
school_merge2.sort_values(by=['overall_passing_pct'], inplace=True, ascending=True)
#Pick top 5
school_merge2[0:5]

,type,size,budget,average_math,average_read,budget_per_student,math_passing_pct,reading_passing_pct,overall_passing_pct
school_name,,,,,,,,,
Figueroa High School,District,2949,"$1,884,411",76.711767,81.158020,639.0,66.0,81.0,40.0
Ford High School,District,2739,"$1,763,916",77.102592,80.746258,644.0,68.0,79.0,40.0
Hernandez High School,District,4635,"$3,022,020",77.289752,80.934412,652.0,67.0,81.0,40.0
Rodriguez High School,District,3999,"$2,547,363",76.842711,80.744686,637.0,66.0,80.0,40.0
Bailey High School,District,4976,"$3,124,928",77.048432,81.033963,628.0,67.0,82.0,41.0


## Math Scores by Grade

In [7]:
# students average math score by school by grade
# how do i sort columns?  9th is showing up after the 10s.

school_table5 = school_data_complete.loc[school_data_complete["budget"] > 0, ["school_name","grade","math_score","gender"]]
school_table5
grouped_school_table5 = school_table5.groupby(["school_name", "grade"]).sum()
grouped_school_table5
# count of students by school by grade

school_table5a = school_data_complete.loc[school_data_complete["budget"] > 0, ["school_name","grade","Student ID"]]
grouped_school_table5a = school_table5a.groupby(["school_name", "grade"]).count()
# merge average math score by school by grade with count of students by school by grade
school_merge_by_grade = grouped_school_table5.merge(grouped_school_table5a, on=["school_name","grade"], how="outer")
school_merge_by_grade["avg_math_score"] = school_merge_by_grade["math_score"]/school_merge_by_grade["Student ID"]

# create a pivot table of math scores averages by grade
school_merge_by_grade = school_merge_by_grade.drop(columns=["math_score","Student ID"])
school_merge_by_grade_pivot = school_merge_by_grade.pivot_table("avg_math_score", "grade", "school_name")
school_merge_by_grade_pivot.sort_values(by=['grade'], inplace=True, ascending=True)

# transpose pivot table to switch columns and rows
school_merge_by_grade_pivot=school_merge_by_grade_pivot.T
# reorder columns
school_merge_by_grade_pivot=school_merge_by_grade_pivot[["9th","10th","11th","12th"]]
school_merge_by_grade_pivot

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
# students average reading score by school by grade

school_table6 = school_data_complete.loc[school_data_complete["budget"] > 0, ["school_name","grade","reading_score","gender"]]
grouped_school_table6 = school_table6.groupby(["school_name", "grade"]).sum()
grouped_school_table6
# count of students by school by grade
school_table6a = school_data_complete.loc[school_data_complete["budget"] > 0, ["school_name","grade","Student ID"]]
grouped_school_table6a = school_table6a.groupby(["school_name", "grade"]).count()
grouped_school_table6a

# merge average math score by school by grade with count of students by school by grade
school_merge_by_grade2 = grouped_school_table6.merge(grouped_school_table6a, on=["school_name","grade"], how="outer")
school_merge_by_grade2
school_merge_by_grade2["avg_reading_score"] = school_merge_by_grade2["reading_score"]/school_merge_by_grade2["Student ID"]
school_merge_by_grade2 = school_merge_by_grade2.drop(columns=["reading_score","Student ID"])

# create a pivot table of reading score averages by grade
school_merge_by_grade2_pivot = school_merge_by_grade2.pivot_table("avg_reading_score", "grade", "school_name")
school_merge_by_grade2_pivot.sort_values(by=['grade'], inplace=True, ascending=True)

# transpose pivot table to switch columns and rows
school_merge_by_grade2_pivot=school_merge_by_grade_pivot.T

# reorder columns
school_merge_by_grade2_pivot=school_merge_by_grade_pivot[["9th","10th","11th","12th"]]
school_merge_by_grade2_pivot

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
# Scores by School Spending Exercise
# Create the bins in which Data will be held
# Bins are 0, 585, 615, 645, 675

spending_bins = [0, 585, 615, 645, 675]

# Create the names for the four bins
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
#merge budget per student with data
school_merge2_copy1["Budget Per Student Summary"] = pd.cut(school_merge2_copy1["budget_per_student"], spending_bins, labels=group_names)
school_merge2_copy1 = school_merge2_copy1.groupby("Budget Per Student Summary")
#determine average math and reading scores by per student budget spending category
performance_by_school_spending = school_merge2_copy1["average_math","average_read"].sum()/school_merge2_copy1["average_math","average_read"].count()
# determine average passing percentages by per student budget spending category
performance_by_school_spending2=school_merge2_copy1["math_passing_pct","reading_passing_pct","overall_passing_pct"].mean()
# merge with average math and reading scores
performance_by_school_spending_merge=performance_by_school_spending.merge(performance_by_school_spending2, on=["Budget Per Student Summary"], how="outer")
performance_by_school_spending_merge
#TRIED TO ELIMINATE DECIMALS for PASSING PCT, BUT COULDN'T PUT ROUNDING AROUND THE FUNCTION MEAN()

,average_math,average_read,math_passing_pct,reading_passing_pct,overall_passing_pct
Budget Per Student Summary,,,,,
<$585,83.455399,83.933814,93.500000,96.75,48.250000
$585-615,83.599686,83.885211,94.500000,96.00,48.000000
$615-645,79.079225,81.891436,75.500000,86.00,43.166667
$645-675,76.997210,81.027843,66.333333,81.00,40.666667


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
# Scores by School Size Exercise
# Create the 3 bins in which Data will be held
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

#merge school size with data
school_merge2_copy2["School Size Summary"] = pd.cut(school_merge2_copy2["size"], size_bins, labels=group_names)
school_merge2_copy2 = school_merge2_copy2.groupby("School Size Summary")
# determine average math and reading scores by per school size category
performance_by_school_size = school_merge2_copy2["average_math","average_read"].sum()/school_merge2_copy2["average_math","average_read"].count()
# determine passing percentages by per school size category
performance_by_school_size2=school_merge2_copy2["math_passing_pct","reading_passing_pct","overall_passing_pct"].mean()
# merge with average math and reading scores
performance_by_school_size_merge=performance_by_school_size.merge(performance_by_school_size2, on=["School Size Summary"], how="outer")
performance_by_school_size_merge
# Same rounding issue as above.

,average_math,average_read,math_passing_pct,reading_passing_pct,overall_passing_pct
School Size Summary,,,,,
Small(<1000),83.821598,83.929843,94.0,96.00,48.000
Medium(1000-2000),83.374684,83.864438,93.4,96.80,48.600
Large(2000-5000),77.746417,81.344493,70.0,82.75,41.375


## Scores by School Type

* Perform the same operations as above, based on school type.

In [11]:
# Scores by School Type Exercise

# group data by type
school_merge_grouped = school_merge2_copy3.groupby("type")
# calculate avg math and avg reading scores by school type
performance_by_school_type = school_merge_grouped["average_math","average_read"].sum()/school_merge_grouped["average_math","average_read"].count()
# calculate passing percentages by school type
performance_by_school_type2=round(school_merge_grouped["math_passing_pct","reading_passing_pct","overall_passing_pct"].mean() * 2 / 2)
# merge with average math and reading scores
performance_by_school_type_merge=performance_by_school_type.merge(performance_by_school_type2, on=["type"], how="outer")
performance_by_school_type_merge

,average_math,average_read,math_passing_pct,reading_passing_pct,overall_passing_pct
type,,,,,
Charter,83.473852,83.896421,94.0,97.0,48.0
District,76.956733,80.966636,67.0,81.0,40.0
